### Import libraries

In [58]:
import pandas as pd
pd.options.display.max_columns = 999
import pgeocode as geo
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

### Import 1990 file

In [59]:
sc= pd.read_excel(r'Input\\SocCapVariables.1990.1997.2005.xls', sheet_name='1990')
sc=sc[sc.fips!=51780] # pop is not available. 51780 exists only in the oldest file (SocCapVariables.1990.1997.2005)
sc.drop(columns=['pvote92'], inplace=True)
sc.head(1)

,fips,AREANAME,bowl90,civic90,fit90,golf90,relig90,sport90,memspt90,pol90,prof90,bus90,labor90,memnec90,assn90,respn90,pvote88,nccs90,pop90,ski90pcm,ski90std
0,1001,"Autauga, AL",0,7,1,0,21,0,1,0,0,2,5,0,37,65.0,48.337755,5,34222,-0.897834,-0.474257


In [60]:
sc.count()

fips        3110
AREANAME    3110
bowl90      3110
civic90     3110
fit90       3110
golf90      3110
relig90     3110
sport90     3110
memspt90    3110
pol90       3110
prof90      3110
bus90       3110
labor90     3110
memnec90    3110
assn90      3110
respn90     2435
pvote88     3110
nccs90      3110
pop90       3110
ski90pcm    3110
ski90std    3110
dtype: int64

- Many missing obs for respn90.
- All variables with 90 suffix.
- Two pvote: pvote88 and pvote92. Following studies, pvote88 is used.

### Rename columns

In [61]:
cols_rename= {
            'AREANAME':'area',
            'ski90pcm':'sk_u',   
            'ski90std': 'sk',
            }

In [62]:
sc.rename(columns=cols_rename, inplace=True)
sc.columns= sc.columns.str.strip().str.lower()
sc.columns=sc.columns.str.replace('\d+','',regex=True).str.strip()
sc.head(1)

,fips,area,bowl,civic,fit,golf,relig,sport,memspt,pol,prof,bus,labor,memnec,assn,respn,pvote,nccs,pop,sk_u,sk
0,1001,"Autauga, AL",0,7,1,0,21,0,1,0,0,2,5,0,37,65.0,48.337755,5,34222,-0.897834,-0.474257


### Calculate nccs based on 1997, 2005, 2009 data
##### Import files

In [63]:
sc.drop(columns='nccs', inplace=True)

In [64]:
# Old files
# sc97= pd.read_excel(r'Input\\SocCapVariables.1990.1997.2005.xls', sheet_name='1997')[['fips','nccs97']]
# sc05= pd.read_excel(r'Input\\SocCapVariables.1990.1997.2005.xls', sheet_name='2005')[['fips','nccs05']]
# sc09= pd.read_excel(r'Input\\social_capital.97-05-09.updated8.28.17.xlsx', sheet_name='2009')[['fips','nccs09']]

# New files / consistent with Hasan et al. (2017)
sc97= pd.read_excel(r'Input\\social_capital.97-05-09.updated8.28.17.xlsx', sheet_name='1997')[['fips','nccs97','pop97']]
sc05= pd.read_excel(r'Input\\social_capital.97-05-09.updated8.28.17.xlsx', sheet_name='2005')[['fips','nccs05','pop05']]
sc09= pd.read_excel(r'Input\\social_capital.97-05-09.updated8.28.17.xlsx', sheet_name='2009')[['fips','nccs09','pop09']]

print(sc97[['nccs97']].mean())
print(sc05[['nccs05']].mean())
print(sc09[['nccs09']].mean())
print("*"*50)

nccs97    353.943226
dtype: float64
nccs05    443.735114
dtype: float64
nccs09    494.433677
dtype: float64
**************************************************


- The mean values consistent with Appendix (p.28) in Hasan et al. (2017)

##### Check missing obs

In [65]:
sc97['nccs97']= sc97['nccs97'] / (sc97['pop97'] / 10000)
sc05['nccs05']= sc05['nccs05'] / (sc05['pop05'] / 10000)
sc09['nccs09']= sc09['nccs09'] / (sc09['pop09'] / 10000)
print(sc97[['nccs97', 'pop97']].count())
print(sc05[['nccs05', 'pop05']].count())
print(sc09[['nccs09', 'pop09']].count())

nccs97    3100
pop97     3108
dtype: int64
nccs05    3107
pop05     3108
dtype: int64
nccs09    3106
pop09     3108
dtype: int64


- Some missing obs from sc97, sc05 and sc09

##### Replace missing values with zero

In [66]:
sc97['nccs97']= sc97['nccs97'].replace(np.nan, 0) # many missing
sc05['nccs05']= sc05['nccs05'].replace(np.nan, 0) # many missing
sc09['nccs09']= sc09['nccs09'].replace(np.nan, 0) # many missing
print(sc97[['nccs97']].count())
print(sc05[['nccs05']].count())
print(sc09[['nccs09']].count())

nccs97    3108
dtype: int64
nccs05    3108
dtype: int64
nccs09    3108
dtype: int64


In [67]:
sc97.head(2)

,fips,nccs97,pop97
0,1001,22.794510,41238
1,1003,31.037768,130164


In [68]:
sc05.head(2)

,fips,nccs05,pop05
0,1001,28.786537,49676
1,1003,36.255341,162183


In [69]:
sc09.head(2)

,fips,nccs09,pop09
0,1001,33.619655,54135
1,1003,41.080008,179406


##### Calculate the nccs growth rate and estimate nccs 1990

In [70]:
nccs97= sc97[['fips', 'nccs97']]
nccs05= sc05[['fips', 'nccs05']]
nccs09= sc09[['fips', 'nccs09']]

nccs= pd.merge(nccs97, nccs05, on='fips')
nccs= pd.merge(nccs, nccs09, on='fips')

nccs['nccs_growth1']=1+(nccs['nccs05']-nccs['nccs97']) / nccs['nccs97']
nccs['nccs_growth2']=1+(nccs['nccs09']-nccs['nccs05']) / nccs['nccs05']
nccs['nccs_growth_avg']= (nccs['nccs_growth1']+nccs['nccs_growth2'])/2
nccs['nccs']= (nccs['nccs97']/nccs['nccs_growth_avg']).round(4) # nccs90 estimated
nccs.head(2)

sc=pd.merge(sc, nccs[['fips','nccs']], on="fips", how= "left")
print(sc[['nccs']].count())

nccs    3107
dtype: int64


In [71]:
sc.head()

,fips,area,bowl,civic,fit,golf,relig,sport,memspt,pol,prof,bus,labor,memnec,assn,respn,pvote,pop,sk_u,sk,nccs
0,1001,"Autauga, AL",0,7,1,0,21,0,1,0,0,2,5,0,37,65.0,48.337755,34222,-0.897834,-0.474257,18.7550
1,1003,"Baldwin, AL",2,29,4,1,80,0,9,1,0,7,3,3,139,65.0,48.933977,98280,-0.362414,-0.195187,26.9755
2,1005,"Barbour, AL",0,2,0,0,14,0,1,0,0,1,0,1,19,60.0,49.573887,25417,-1.499349,-0.767512,34.5457
3,1007,"Bibb, AL",0,0,0,0,14,0,1,0,0,0,1,0,16,67.0,43.673088,16576,-0.930472,-0.446631,21.2694
4,1009,"Blount, AL",0,0,0,2,15,0,0,0,0,0,5,0,22,69.0,46.378201,39248,-1.304095,-0.656223,14.1055


### Check missing obs of respn and nccs and repalce them with zero

In [72]:
sc[['respn']].count() # many missing

respn    2435
dtype: int64

In [73]:
sc[['nccs']].count() # many missing

nccs    3107
dtype: int64

In [74]:
sc['respn']= sc['respn'].replace(np.nan, 0) # valid only for 2435. missing for (3110-2435). This issue only for 1990.
sc['nccs']= sc['nccs'].replace(np.nan, 0) # missing for about 10~20 obs. But it affects results significantly.

In [75]:
sc[['nccs','respn']].count()

nccs     3110
respn    3110
dtype: int64

### Calculate assn based on 10 common categories following Hasan et al. (2017), etc.

In [76]:
sc['assn']= (sc['relig'] + sc['civic'] + sc['bus'] + sc['pol'] + sc['prof'] + sc['labor'] + sc['bowl'] + sc['fit'] + sc['golf'] + sc['sport']) / (sc['pop']/1000)

sc.drop(columns=['bowl', 'civic', 'fit', 'golf', 'relig', 'sport', 'pol', 'prof', 'bus', 'labor'], inplace=True)
sc.drop(columns=['memspt','memnec'], inplace=True)
sc.drop(columns=['pop'],inplace=True)
sc.head(3)

,fips,area,assn,respn,pvote,sk_u,sk,nccs
0,1001,"Autauga, AL",1.051955,65.0,48.337755,-0.897834,-0.474257,18.7550
1,1003,"Baldwin, AL",1.292226,65.0,48.933977,-0.362414,-0.195187,26.9755
2,1005,"Barbour, AL",0.668844,60.0,49.573887,-1.499349,-0.767512,34.5457


In [77]:
sc.count()

fips     3110
area     3110
assn     3110
respn    3110
pvote    3110
sk_u     3110
sk       3110
nccs     3110
dtype: int64

- no missing obs

### Scaling issues (to be consistent with other datasets)

In [78]:
sc['respn']= sc['respn']/100
sc['pvote']= sc['pvote']/100

### Principal Component Analysis

In [79]:
x = sc.loc[:, ['respn','pvote','assn','nccs']].values
# Separating out the target
y = sc.loc[:,['fips']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

pca = PCA(n_components=1)
principalComponents = pca.fit_transform(x)
pca_df = pd.DataFrame(data = principalComponents
             , columns = ['pc1'])
pca_df.head(2)

,pc1
0,-1.108387
1,-0.701351


### Merge the first PCA component with Original SC set

In [80]:
sc['pc1']=pca_df[['pc1']]

In [81]:
sc.head()

,fips,area,assn,respn,pvote,sk_u,sk,nccs,pc1
0,1001,"Autauga, AL",1.051955,0.65,0.483378,-0.897834,-0.474257,18.7550,-1.108387
1,1003,"Baldwin, AL",1.292226,0.65,0.489340,-0.362414,-0.195187,26.9755,-0.701351
2,1005,"Barbour, AL",0.668844,0.60,0.495739,-1.499349,-0.767512,34.5457,-0.971752
3,1007,"Bibb, AL",0.904923,0.67,0.436731,-0.930472,-0.446631,21.2694,-1.438512
4,1009,"Blount, AL",0.560538,0.69,0.463782,-1.304095,-0.656223,14.1055,-1.775053


##### Check the correlation with sk: Hasan et al. (2017), in footnote 5, show that their corr is 0.93.

In [82]:
sc['sk'].corr(sc['pc1'], method='pearson')

0.7887045494506105

In [83]:
sc.shape

(3110, 9)

In [84]:
sc[['pc1']].count()

pc1    3110
dtype: int64

In [85]:
sc['year']=1990
sc.to_csv("output\\sc90.csv")